In [1]:
%matplotlib notebook

In [2]:
import os
import sys
import numpy as np

In [3]:
import imageio
from IPython.display import HTML

In [4]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc, ticker
from mpl_toolkits.mplot3d.axes3d import Axes3D
plt.ioff()

In [5]:
import seaborn as sns
sns.set()

In [6]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
from regression_models import PolynomialRegression

## Data Preparation

### Gathering

$D$ is a dictionary in the form of $D[(run, visdepth, timestep, \alpha, \beta)] = performance$

where:

$\alpha$ = learning rate

$\beta$ = discount rate

In [8]:
D = {}

In [9]:
max_vis_depth = 4
num_runs = 10
step_size = 10

In [10]:
for depth in range(1, max_vis_depth + 1):
    for run in range(1, num_runs + 1):
        dir_input = "sims/cat_mouse/data/" + str(depth) + "/data" + str(run) + ".txt"
        with open(os.path.abspath(os.path.join('..', dir_input))) as f:
            for line in f.readlines():
                line = list(map(int, line.split()))
                for i in range(2, 12):
                    D[(run, depth, (i - 1) * step_size, line[0], line[1])] = line[i]
                    # key_temp = (depth, (i - 1) * step_size, line[0], line[1])
                    # if key_temp not in D:
                    #     D[key_temp] = line[i] / num_runs
                    # else:
                    #    D[key_temp] += line[i] / num_runs

In [11]:
n = 11
def_depth = 1
def_timestep = 100

In [12]:
X_domain = np.linspace(0, 1, n)
X = np.zeros((4, n, n))
X[0] = np.full((n, n), def_depth)
X[1] = np.full((n, n), def_timestep)
X[2], X[3] = np.meshgrid(X_domain, X_domain)

In [13]:
Y = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        Y[i][j] = sum(D[(run, def_depth, def_timestep, i, j)] for run in range(1, num_runs + 1)) / num_runs

In [14]:
y_min = np.amin(Y)
print('y_min:', y_min)
y_max = np.amax(Y)
print('y_max:', y_max)
y_range = y_max - y_min
print('y_range:', y_range)
print('offset:', y_range/2)

y_min: 4646.3
y_max: 14441.0
y_range: 9794.7
offset: 4897.35


## Data Visualization

### Regression

In [15]:
model = PolynomialRegression(X, Y)

#### Linear model

In [16]:
lin_model = model.process(degree=1)
lin_model.print_func()

0.443282*1 + 0.443282*x0 + 44.328163*x1 + 1903.112397*x2 + 3777.065289*x3


In [17]:
print(lin_model.r_squared)

0.417901783681


#### Quadratic model

In [18]:
quad_model = model.process(degree=2)
quad_model.print_func()

0.000076*1 + 0.000076*x0 + 0.007593*x1 + -0.729022*x2 + -0.850853*x3 + 0.000076*x0^2 + 0.007593*x0*x1 + -0.729022*x0*x2 + -0.850853*x0*x3 + 0.759343*x1^2 + -72.902232*x1*x2 + -85.085262*x1*x3 + 9500.127146*x2^2 + -610.666942*x2*x3 + 12592.626616*x3^2


In [19]:
print(quad_model.r_squared)

0.872194916977


#### Cubic model

In [20]:
cube_model = model.process(degree=3)
cube_model.print_func()

0.000000*1 + 0.000000*x0 + 0.000001*x1 + -0.000015*x2 + -0.000093*x3 + -0.000000*x0^2 + 0.000001*x0*x1 + -0.000015*x0*x2 + -0.000093*x0*x3 + 0.000071*x1^2 + -0.001530*x1*x2 + -0.009298*x1*x3 + 0.108501*x2^2 + -0.743440*x2*x3 + 1.618774*x3^2 + 0.000000*x0^3 + 0.000001*x0^2*x1 + -0.000015*x0^2*x2 + -0.000093*x0^2*x3 + 0.000071*x0*x1^2 + -0.001530*x0*x1*x2 + -0.009298*x0*x1*x3 + 0.108501*x0*x2^2 + -0.743440*x0*x2*x3 + 1.618774*x0*x3^2 + 0.007048*x1^3 + -0.152960*x1^2*x2 + -0.929839*x1^2*x3 + 10.850129*x1*x2^2 + -74.343952*x1*x2*x3 + 161.877399*x1*x3^2 + 2410.946890*x2^3 + 9596.953804*x2^2*x3 + -2771.738716*x2*x3^2 + -1474.987648*x3^3


In [21]:
print(cube_model.r_squared)

0.891426037679


### Plotting

In [22]:
def plot_animate_preprocess(model, degree):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    
    p = model.plot(ax, X[2], X[3], Y, idx=(2, 3), wireframe=True, show_contours=False)
    q = model.plot(ax, X[2], X[3], model.process(degree=degree).predict(), 
                   idx=(2,3), lims=((-0.5, 1.0), (0.0, 1.5), (0, 15000)),
                   show_labels=True, labels=("Learning Rate", "Discount Rate", "Agent Performance"))

    formatter = ticker.ScalarFormatter(useMathText=True)
    formatter.set_scientific(True) 
    formatter.set_powerlimits((-1,1)) 
    ax.zaxis.set_major_formatter(formatter)

    fig.patch.set_alpha(0.)
    ax.patch.set_alpha(0.0)
    ax.grid(False)
    
    for ii in range(0, 360, 1):
        ax.view_init(elev=30., azim=ii)
        plt.savefig("perf_vs_params/%d/frame%d.jpeg" % (degree, ii), transparent=True)

In [23]:
def plot_animate_beta(degree, secs=6, fps=60):
    frame_steps = 360 // (secs * fps)
    
    images = []
    for ii in range(0, 360, frame_steps):
        images.append(imageio.imread("perf_vs_params/%d/frame%d.jpeg" % (degree,ii)))
    imageio.mimsave("perf_vs_params/%d/anim%d.gif" % (degree, fps), images, fps=60)
    
    return HTML('<img src="perf_vs_params/%d/anim%d.gif">' % (degree, fps))

#### Linear Estimate

In [24]:
plot_animate_preprocess(model, degree=1)

In [25]:
plot_animate_beta(degree=1, secs=6, fps=60)

#### Quadratic Estimate

In [26]:
plot_animate_preprocess(model, degree=2)

In [27]:
plot_animate_beta(degree=2, secs=6, fps=60)

#### Cubic Estimate

In [28]:
plot_animate_preprocess(model, degree=3)

In [29]:
plot_animate_beta(degree=3, secs=6, fps=60)